# Distraction Detection Training

Training YOLOv8 classification model for distracted driving detection

Model: yolov8n-cls
Image size: 224
Epochs: 10
Batch: 32

In [ ]:
MODEL_SIZE = "n"
IMG_SIZE = 224
EPOCHS = 10
BATCH_SIZE = 32
DATA_DIR = "../data/processed/distraction_binary"
OUTPUT_DIR = "../models/distraction_yolov8"
BEST_MODEL_PATH = f"{OUTPUT_DIR}/best.pt"

print(f"Training: yolov8{MODEL_SIZE}-cls")
print(f"Image size: {IMG_SIZE}, Epochs: {EPOCHS}, Batch: {BATCH_SIZE}")

Training: yolov8n-cls
Image size: 224, Epochs: 10, Batch: 32


In [10]:
from ultralytics import YOLO
from pathlib import Path
import torch

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch: 2.9.1
CUDA available: False


In [ ]:
model = YOLO(f"yolov8{MODEL_SIZE}-cls.pt")

Model loaded


In [ ]:
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

results = model.train(
    data=DATA_DIR,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    project=OUTPUT_DIR,
    name=f"yolov8{MODEL_SIZE}_imgsz{IMG_SIZE}_ep{EPOCHS}",
    exist_ok=True
)

print("Training complete")

New https://pypi.org/project/ultralytics/8.4.8 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.7 🚀 Python-3.13.7 torch-2.9.1 CPU (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../configs/distraction_binary.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolov8n_imgsz224_ep10, nbs=64, nms=Fals

RuntimeError: Dataset '../configs/distraction_binary.yaml' error ❌ Classification datasets must be a directory (data="path/to/dir") not a file (data="../configs/distraction_binary.yaml"), See https://docs.ultralytics.com/datasets/classify/

In [ ]:
# find best model
possible_paths = [
    f"{OUTPUT_DIR}/yolov8{MODEL_SIZE}_imgsz{IMG_SIZE}_ep{EPOCHS}/weights/best.pt",
    f"../runs/classify/{OUTPUT_DIR}/yolov8{MODEL_SIZE}_imgsz{IMG_SIZE}_ep{EPOCHS}/weights/best.pt",
    f"../runs/classify/models/distraction_yolov8/yolov8{MODEL_SIZE}_imgsz{IMG_SIZE}_ep{EPOCHS}/weights/best.pt",
]

best_model_path = None
for path in possible_paths:
    if Path(path).exists():
        best_model_path = path
        break

if not best_model_path:
    found = list(Path("..").rglob(f"**/yolov8{MODEL_SIZE}_imgsz{IMG_SIZE}_ep{EPOCHS}/weights/best.pt"))
    if found:
        best_model_path = str(found[0])

if best_model_path:
    best_model = YOLO(best_model_path)
    import shutil
    Path(BEST_MODEL_PATH).parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(best_model_path, BEST_MODEL_PATH)
else:
    if Path(BEST_MODEL_PATH).exists():
        best_model = YOLO(BEST_MODEL_PATH)
    else:
        raise FileNotFoundError("Best model not found")

# evaluate
val_results = best_model.val(data=DATA_DIR, split='val', verbose=False)
test_results = best_model.val(data=DATA_DIR, split='test', verbose=False)

print(f"Val accuracy: {val_results.top1:.4f}")
print(f"Test accuracy: {test_results.top1:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

test_dir = Path(DATA_DIR) / "test"
class_names = ["not_distracted", "distracted"]

test_images = []
true_labels = []
pred_labels = []

for class_idx, class_name in enumerate(class_names):
    class_dir = test_dir / class_name
    if class_dir.exists():
        for img_path in class_dir.glob("*.jpg"):
            test_images.append(str(img_path))
            true_labels.append(class_idx)

print(f"Found {len(test_images)} test images")

for i, img_path in enumerate(test_images):
    if i % 100 == 0:
        print(f"Processing {i}/{len(test_images)}...")
    
    results = best_model.predict(img_path, verbose=False)
    pred_class_idx = int(results[0].probs.top1)
    pred_labels.append(pred_class_idx)

cm = confusion_matrix(true_labels, pred_labels)

plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix', fontsize=14)
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
                fontsize=12)

plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()

confusion_matrix_path = f"{OUTPUT_DIR}/confusion_matrix.png"
plt.savefig(confusion_matrix_path, dpi=150, bbox_inches='tight')
print(f"Saved to: {confusion_matrix_path}")
plt.show()

print(classification_report(true_labels, pred_labels, target_names=class_names))